In [158]:
from AOC_utils import get_day
get_day(3)

with open('days/day3input.txt') as f:
    input_data = f.readlines()
input_data = [line[:-1] for line in input_data] # Remove \n
input_data[:7]

['.....613...................................439............498.........................438......617....343.............942...................',
 '.......*............790...........269..735..*........................../679..............*444.*.........*.......147...*.............441.....',
 '....539......422.......*......662*........*..691..........*124.15..675.................=.......404...872............237......930.....+......',
 '........334.............861.........%....479..........424.......+.@.......402.......314...905................833........*293................',
 '..........#.......................906.................*...............950.....................712...437.........*142.359........551.14......',
 '....509.....=...........890...................&........9................./..847.154..568............@...102................280...*..........',
 '.....*..950.67.............-......161.......530....=...................=............*...../..............@......................

In [18]:
example = '''
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
'''
example = example.split('\n')
example = example[1:-1]

In [ ]:
import numpy as np

In [156]:
def parse_data(data):
    data = np.array([list(row) for row in data])

    # add a border of '.' around the data to make indexing less annoying
    data = np.concatenate((np.array([['.'] * len(data[0])]), data, np.array([['.'] * len(data[0])])), axis=0)
    data = np.concatenate((np.array([['.'] * len(data.T[0])]).reshape(-1,1), data, np.array([['.'] * len(data.T[0])]).reshape(-1,1)), axis=1)

    # get symbols
    symbols = np.unique(data)
    # remove numbers from symbols
    symbols = symbols[[not symbol.isnumeric() for symbol in symbols]]
    # remove '.' from symbols
    symbols = symbols[symbols != '.']
    print(symbols)


    total = 0
    for i in range(1, len(data)-1):
        whole_number = ''
        near_symbol = False
        for j in range(1, len(data[i])-1):

            # search for and construct whole numbers, and check for nearby symbols
            if data[i][j].isnumeric():
                whole_number += data[i][j]

                # check surrounding indices for symbols (maybe could have used convolutions?)
                surround = np.array([
                    [data[i-1][j-1], data[i-1][j], data[i-1][j+1]],
                    [data[i][j-1], data[i][j], data[i][j+1]],
                    [data[i+1][j-1], data[i+1][j], data[i+1][j+1]]
                ])
                # check if any of the surrounding indices are symbols
                for point in surround.flatten():
                    if point in symbols:
                        near_symbol = True

            # if we aren't at a number, then we need to check if we have a whole number to add to the total
            else:
                if len(whole_number) > 0:
                    whole_number = int(whole_number)
                    if near_symbol:
                        total += whole_number
                    whole_number = ''
                    near_symbol = False
        
        # This edge case cost me so much time
        # when we get to the end of a row, we need to check if we have a whole number to add to the total
        # because if the line ends with the number, then we wouldn't have checked it using the if else above
        if len(whole_number) > 0:
            whole_number = int(whole_number)
            if near_symbol:
                total += whole_number
            whole_number = ''
            near_symbol = False
        # in hindsight, simply padding another '.' around the data would have avoided this case
        # might be worth remembering that for future problems

    return total

In [157]:
parse_data(input_data)

['#' '$' '%' '&' '*' '+' '-' '/' '=' '@']


527144

In [144]:
# helper function that takes indices of a number, and uses surrounding indices to get the whole number
def get_full_number(data, i, j):
    whole_number = data[i][j]

    if data[i][j-1].isnumeric():
        whole_number = data[i][j-1] + whole_number
        if j-2 >= 0 and data[i][j-2].isnumeric():
            whole_number = data[i][j-2] + whole_number
    if data[i][j+1].isnumeric():
        whole_number = whole_number + data[i][j+1]
        if j+2 < (len(data[i])) and data[i][j+2].isnumeric():
            whole_number = whole_number + data[i][j+2]
    
    return whole_number

def parse_data_part2(data):
    data = np.array([list(row) for row in data])

    # add a border of '.' around the data to make indexing less annoying
    data = np.concatenate((np.array([['.'] * len(data[0])]), data, np.array([['.'] * len(data[0])])), axis=0)
    data = np.concatenate((np.array([['.'] * len(data.T[0])]).reshape(-1,1), data, np.array([['.'] * len(data.T[0])]).reshape(-1,1)), axis=1)

    total = 0
    for i in range(1, len(data)-1):
        for j in range(1, len(data[i])-1):

            # find gears
            if data[i][j] == '*':
                
                gears = []
                # we need to check the surrounding points for numbers
                if data[i][j-1].isnumeric():
                    whole_number = get_full_number(data, i, j-1) # helper function
                    gears.append(whole_number)
                if data[i][j+1].isnumeric():
                    whole_number = get_full_number(data, i, j+1)
                    gears.append(whole_number)
                if data[i-1][j].isnumeric():
                    whole_number = get_full_number(data, i-1, j)
                    gears.append(whole_number)
                if data[i+1][j].isnumeric():
                    whole_number = get_full_number(data, i+1, j)
                    gears.append(whole_number)
                if data[i-1][j-1].isnumeric():
                    whole_number = get_full_number(data, i-1, j-1)
                    gears.append(whole_number)
                if data[i-1][j+1].isnumeric():
                    whole_number = get_full_number(data, i-1, j+1)
                    gears.append(whole_number)
                if data[i+1][j-1].isnumeric():
                    whole_number = get_full_number(data, i+1, j-1)
                    gears.append(whole_number)
                if data[i+1][j+1].isnumeric():
                    whole_number = get_full_number(data, i+1, j+1)
                    gears.append(whole_number)

                gears = [int(gear) for gear in gears]

                # remove duplicates
                gears = list(set(gears))
                # this was risky, there's no constraint that two adjacent gears couldn't be the same number
                # but the odds were small, and it worked for my input
                # if this didn't work, we would have had to track a second list for the indices of the gears
                # and use that to check whether we had already added a number

                # only add to total if we have exactly two gears
                if len(gears) == 2:
                    total += gears[0] * gears[1]
                    
    return total

In [147]:
parse_data_part2(example)

467835

In [148]:
parse_data_part2(input_data)

81463996